In [1]:
import mechanicalsoup
import pandas as pd
import numpy as np
import datetime

In [2]:
def month_to_string(monthinteger):
    """
    Input: Month integer value
    Output: Month string value
    
    Taken from stackoverflow, of course: https://stackoverflow.com/questions/6557553/get-month-name-from-number
    """
    return datetime.date(1900, monthinteger, 1).strftime('%B')

In [3]:
def scrape_pollutant_sefton(pollutant,period,date,url):
    no_data = False   #Boolean to check whether there is "no data".
    missing_info = False #Boolean to check whether there is enough info 
    
    date_list = []
    month_list = []
    hour_list = []
    pollutant_list = []
    
    
    allowed_years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019'] #List of allowed years.
    allowed_pollutants = ['NO','NO2','NOx','PM10BAM']
    allowed_periods = ["hourly_day","hourly_week","hourly_month","daily_week","daily_month","daily_year","hourly_year"]
    period_key = {
        "hourly_day": "1",
        "hourly_week": "2",
        "hourly_month": "3",
        "daily_week": "4",
        "daily_month": "5",
        "daily_year": "6",
        "hourly_year": "7"
    }
    month_hours_key = {
        "1": 744,
        "2": 672,
        "3": 744,
        "4": 720,
        "5": 744,
        "6": 720,
        "7": 744,
        "8": 744,
        "9": 720,
        "10": 744,
        "11": 720,
        "12": 744
    }
    
    ### Convert variables into making sense for form input ###
    date = date.split("/")
    day = date[0] #Day
    month = date[1] #Month
    print("Month: ", month)
    year = date[2] #Year
    
    #Get rid of 0 in day if 01-09
    if day[0] == '0':
        day = day.replace('0','')
    
    #Get rid of 0 in month if 01-09
    if month[0] == '0':
        month = month.replace('0','')
    
    #Make sure year is allowed value
    if year not in allowed_years:
        return "Year not allowed. Must be between 2010-2019. Double check input."
    
    #Make sure pollutant is allowed value
    if pollutant not in allowed_pollutants:
        return "Pollutant not allowed. Must be either 'NO','NO2','NOx','PM10BAM'. Double check input."
    
    #Make sure period is allowed value
    if period not in allowed_periods:
        return "Period not allowed. Must be either 'hourly_day','hourly_week','hourly_month','daily_week','daily_month','daily_year','hourly_year'. Double check input."
    
    
    browser = mechanicalsoup.StatefulBrowser()
    
    browser.open(url)

    browser.select_form('form[id="form1"]')

    browser["ShortName"] = pollutant
    browser["Choice"] = period_key[period]
    browser["StartDay"] = day
    browser["StartMonth"] = month
    browser["StartYear"] = year

    response = browser.submit_selected()

    #return print(response.text)
    soup = browser.get_current_page()
    
    #for result in soup.findAll(class_='GridAltStyle'):
    counter = 0 #Counter used to know where to split the time
    for result in soup.findAll('tr'):
        #result = result.replace('<tr class="GridAltStyle"><td>','')
        result = result.text  #Convert to text
        
        #split at year and replace year in correct place
        if year in result:
            result = result.split("/"+year) 

            if len(result) > 1:
                result[0] = result[0]+"/"+year
                Date = result[0]
                
                if len(result[1])>1:

                    if len(result[1])> 1 and counter < 10:
                        #print("Counter below 10")
                        #print("result[1][0]: ", result[1][0])
                        #if result[1][0]=="1" or result[1][0]=="2":
                        Time = result[1][0]
                        Pollutant = float(result[1][1:].strip())

                        #print("Date: ",Date)
                        #print("Time: ",Time)
                        #print("Pollutant: ",Pollutant)

                    elif len(result[1])> 1 and counter >= 10:
                        #print("Counter over or eq 10")
                        #print("result[1][0]: ", result[1][0])
                        #if result[1][0]=="1" or result[1][0]=="2":
                        Time = result[1][0] + result[1][1]
                        Pollutant = float(result[1][2:].strip())

                        #print("Date: ",Date)
                        #print("Time: ",Time)
                        #print("Pollutant: ",Pollutant)

                    
                    
                    #print("Counter: ",counter)           
                    counter+=1


                    if int(Time)==23:
                        counter = 0
                      
                    #Append to list before putting into dataframe
                    date_list.append(Date)
                    month_list.append(month)
                    hour_list.append(Time)
                    pollutant_list.append(Pollutant)
                    
    d = {"Date": date_list, "Month": month_list, "Hour Taken":hour_list, "Pollutant": pollutant_list}
    
    df = pd.DataFrame(d,columns=["Date","Month","Hour Taken","Pollutant"])
    
    print("Number of rows: ", df.shape[0])
    print("month_hours_key[month]: ", month_hours_key[month])
    print("Does df.shape[0] != month_hours_key[month]?: ", df.shape[0] != month_hours_key[month])
    
    #Check if there is no data at all. i.e "There is no data available for those parameters".
    if df.shape[0] == 0:
        for result_fail in soup.findAll('h2'):
            if "There is no data available for those parameters" in result_fail.text:
                print("!!!!!!!!!!!!!!!!!!!!!!!")
                print(result_fail.text)
                print("!!!!!!!!!!!!!!!!!!!!!!!")
                no_data = True
                
    #Check whether or not there is the correct amount of hours in the month
    if no_data == False and period == "hourly_month" and df.shape[0] != month_hours_key[month]:
        if month == "02" and int(year)%4 == 0:  #Take into account leap years
            missing_info = True
        elif month == "02" and int(year)%4!=0 and df.shape[0] != 696:  #Take into account leap years
            missing_info = True
        else:
            missing_info = True
    
    return df, no_data, missing_info
                    
    #print("date: ",date_list)
    #print("month: ",month_list)
    #print("hour: ",hour_list)
    #print("pollutant: ",pollutant_list)

                        
                        
                    

In [8]:
#Test function
pollutants_list = ['NO','NO2','NOx','PM10BAM']
year_list = ['2010','2011','2012','2013','2014','2015','2016','2017','2018','2019']
month_list = np.linspace(1,12,12).astype('int')


for pollutant_str in pollutants_list:
    print("#####################################################")
    print("New Pollutant: ",pollutant_str)
    print("#####################################################")
    
    for year_val in year_list:

        print("#####################################################")
        print("New Year: ",year_val)
        print("#####################################################")

        for month_val in month_list:
            if month_val < 10:
                month_val = "0" + str(month_val)
            #Input variables
            day = "01"
            month = str(month_val)
            year = year_val #Allowed years: '2010','2011','2012','2013','2014','2015','2016','2017','2018','2019'

            pollutant = "NO"     #Either 'NO','NO2','NOx','PM10BAM'
            period = "hourly_month"   #"Either hourly_day","hourly_week","hourly_month","daily_week","daily_month","daily_year","hourly_year"
            date = day+"/"+month+"/"+year
            url = "http://breathingspace.sefton.gov.uk/Default.aspx?bsPage=air_pollution&option=4&step=2&MonId=2479"

            month_string = month_to_string(int(month))
            print("Date: ",day+" "+month_string+" "+year)
            date_file = year+month+day  #Order the date for the filename

            #Returns the dataframe and whether there is no data or not
            dataframe, no_data, missing_info = scrape_pollutant_sefton(pollutant,period,date,url)

            if no_data == False and missing_info == False:
                filename_csv = "data/"+str(pollutant)+"/complete_set/"+str(pollutant)+"_"+str(period)+"_"+str(month_string)+"_"+str(date_file)+".csv"
                dataframe.to_csv(filename_csv,index=False)
            elif no_data == False and missing_info == True: 
                filename_csv = "data/"+str(pollutant)+"/missing_info/"+str(pollutant)+"_"+str(period)+"_"+str(month_string)+"_"+str(date_file)+"_MISSING_INFO"+".csv"
                dataframe.to_csv(filename_csv,index=False)        
            else:
                filename_csv = "data/"+str(pollutant)+"/empty_data/"+str(pollutant)+"_"+str(period)+"_"+str(month_string)+"_"+str(date_file)+"_NO_DATA_AVAILABLE"+".csv"
                dataframe.to_csv(filename_csv,index=False)




            print("Completed!")
            print("File saved in "+"data/"+str(pollutant)+"_"+str(period)+"_"+str(month_string)+"_"+str(date_file)+".csv")
            print("############################################################################")

print("    ")
print("    ")
print("Script completed :-)")
print("    ")
print("    ")

#####################################################
New Pollutant:  NO
#####################################################
#####################################################
New Year:  2010
#####################################################
Date:  01 January 2010
Month:  01
Number of rows:  0
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  True
!!!!!!!!!!!!!!!!!!!!!!!
There is no data available for those parameters. Please try another set.
!!!!!!!!!!!!!!!!!!!!!!!
Completed!
File saved in data/NO_hourly_month_January_20100101.csv
############################################################################
Date:  01 February 2010
Month:  02
Number of rows:  0
month_hours_key[month]:  672
Does df.shape[0] != month_hours_key[month]?:  True
!!!!!!!!!!!!!!!!!!!!!!!
There is no data available for those parameters. Please try another set.
!!!!!!!!!!!!!!!!!!!!!!!
Completed!
File saved in data/NO_hourly_month_February_20100201.csv
############################

Number of rows:  744
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_March_20120301.csv
############################################################################
Date:  01 April 2012
Month:  04
Number of rows:  719
month_hours_key[month]:  720
Does df.shape[0] != month_hours_key[month]?:  True
Completed!
File saved in data/NO_hourly_month_April_20120401.csv
############################################################################
Date:  01 May 2012
Month:  05
Number of rows:  744
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_May_20120501.csv
############################################################################
Date:  01 June 2012
Month:  06
Number of rows:  720
month_hours_key[month]:  720
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_June_20120601.csv
############

Number of rows:  744
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_August_20140801.csv
############################################################################
Date:  01 September 2014
Month:  09
Number of rows:  711
month_hours_key[month]:  720
Does df.shape[0] != month_hours_key[month]?:  True
Completed!
File saved in data/NO_hourly_month_September_20140901.csv
############################################################################
Date:  01 October 2014
Month:  10
Number of rows:  744
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_October_20141001.csv
############################################################################
Date:  01 November 2014
Month:  11
Number of rows:  720
month_hours_key[month]:  720
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_November_

Number of rows:  677
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  True
Completed!
File saved in data/NO_hourly_month_January_20170101.csv
############################################################################
Date:  01 February 2017
Month:  02
Number of rows:  672
month_hours_key[month]:  672
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_February_20170201.csv
############################################################################
Date:  01 March 2017
Month:  03
Number of rows:  562
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  True
Completed!
File saved in data/NO_hourly_month_March_20170301.csv
############################################################################
Date:  01 April 2017
Month:  04
Number of rows:  0
month_hours_key[month]:  720
Does df.shape[0] != month_hours_key[month]?:  True
!!!!!!!!!!!!!!!!!!!!!!!
There is no data available for those parame

Number of rows:  672
month_hours_key[month]:  672
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_February_20190201.csv
############################################################################
Date:  01 March 2019
Month:  03
Number of rows:  743
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  True
Completed!
File saved in data/NO_hourly_month_March_20190301.csv
############################################################################
Date:  01 April 2019
Month:  04
Number of rows:  672
month_hours_key[month]:  720
Does df.shape[0] != month_hours_key[month]?:  True
Completed!
File saved in data/NO_hourly_month_April_20190401.csv
############################################################################
Date:  01 May 2019
Month:  05
Number of rows:  744
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_May_20190501.csv
########

...

Number of rows:  163
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  True
Completed!
File saved in data/NO_hourly_month_July_20190701.csv
############################################################################
Date:  01 August 2019
Month:  08
Number of rows:  0
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  True
!!!!!!!!!!!!!!!!!!!!!!!
There is no data available for those parameters. Please try another set.
!!!!!!!!!!!!!!!!!!!!!!!
Completed!
File saved in data/NO_hourly_month_August_20190801.csv
############################################################################
Date:  01 September 2019
Month:  09
Number of rows:  0
month_hours_key[month]:  720
Does df.shape[0] != month_hours_key[month]?:  True
!!!!!!!!!!!!!!!!!!!!!!!
There is no data available for those parameters. Please try another set.
!!!!!!!!!!!!!!!!!!!!!!!
Completed!
File saved in data/NO_hourly_month_September_20190901.csv
#####################################

Number of rows:  744
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_July_20110701.csv
############################################################################
Date:  01 August 2011
Month:  08
Number of rows:  744
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_August_20110801.csv
############################################################################
Date:  01 September 2011
Month:  09
Number of rows:  718
month_hours_key[month]:  720
Does df.shape[0] != month_hours_key[month]?:  True
Completed!
File saved in data/NO_hourly_month_September_20110901.csv
############################################################################
Date:  01 October 2011
Month:  10
Number of rows:  744
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_October_201110

Number of rows:  707
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  True
Completed!
File saved in data/NO_hourly_month_December_20131201.csv
############################################################################
#####################################################
New Year:  2014
#####################################################
Date:  01 January 2014
Month:  01
Number of rows:  744
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_January_20140101.csv
############################################################################
Date:  01 February 2014
Month:  02
Number of rows:  672
month_hours_key[month]:  672
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_February_20140201.csv
############################################################################
Date:  01 March 2014
Month:  03
Number of rows:  743
month_hours_key

Number of rows:  698
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  True
Completed!
File saved in data/NO_hourly_month_May_20160501.csv
############################################################################
Date:  01 June 2016
Month:  06
Number of rows:  720
month_hours_key[month]:  720
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_June_20160601.csv
############################################################################
Date:  01 July 2016
Month:  07
Number of rows:  744
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_July_20160701.csv
############################################################################
Date:  01 August 2016
Month:  08
Number of rows:  744
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_August_20160801.csv
##########

Number of rows:  720
month_hours_key[month]:  720
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_June_20180601.csv
############################################################################
Date:  01 July 2018
Month:  07
Number of rows:  744
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_July_20180701.csv
############################################################################
Date:  01 August 2018
Month:  08
Number of rows:  744
month_hours_key[month]:  744
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_August_20180801.csv
############################################################################
Date:  01 September 2018
Month:  09
Number of rows:  720
month_hours_key[month]:  720
Does df.shape[0] != month_hours_key[month]?:  False
Completed!
File saved in data/NO_hourly_month_September_20180901.cs

In [58]:
#Need to also take into account when there no data and what to do. Maybe just put "No data" in pollutant section
year = "2012"
int(year)%4 == 0

True